### Manipulation of Comet database search results of ETNP 2017 P2 samples LC-MS/MS data using python.

Starting with:

    Comet search results (.csv) of PTM-optimized database searches
    These were all searched with 15 ppm precursor tolerance and 0.5 ppm fragement ion tolerance
    Search database included marine fungi and labrinthulomyces discovered using de novo peptide sequencing
    XInteract file includes precursor intensities and protein descriptions

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use for a different file:

#### 1. Change the input file name in *IN 4*
#### 2. Use 'find + replace' (Esc + F) to replace the running # (e.g., 233) for another
#### 3. Update the NAAF factor calculated in *IN 6* into *IN 7*

We don't have technical duplicates here, sadly, unlike the MED4 Pro samples. I exported Comet search results after running through XInteract and saving as `.xlsx` files (pandas really doesn't like to read the `xls` verions of the XInteract output becase there are so many characters in the `protein` column for these samples) into my ETNP 2017 git repo:

In [1]:
cd /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI


In [2]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [3]:
ls

231/  233/  243/  273/  278/  378/


In [4]:
cd 243/

/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/243


In [5]:
ls

ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_Comet15_1FDR_NAAF.csv
ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_Comet15_1FDR_NAAF_totals.csv
ETNP-SKQ17-243-965m-0.3-JA14_PTM-opt_Comet15_1FDR_stripped_peptides.fas
ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_Comet15_1FDR_stripped_peptides.txt
ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_Comet15_1FDR_totals.csv
ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_Comet15_fungidb.csv
JA14_15ppm_quant.xlsx


## 243: 965 m McLane pump filtered on 0.3 um GF-75

In [6]:
# read the CSV into a dataframe using the pandas read_excel function
#cometdup243 = pd.read_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTMopt-hyroxylation/JA14_PTMopt_interact_quant_nopro.pep.csv", index_col='spectrum')

cometdup243 = pd.read_excel("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/243/JA14_15ppm_quant.xlsx")

# remove redundant rows
comet243 = pd.DataFrame.drop_duplicates(cometdup243)

print("# redundant Comet peptides in combined dataframe", len(cometdup243))
print("# nonredundant Comet peptides in combined dataframe", len(comet243))

#look at the dataframe
comet243.head()

# redundant Comet peptides in combined dataframe 13311
# nonredundant Comet peptides in combined dataframe 13311


,spectrum,xcorr,deltacn,expect,ions,peptide,calc_neutral_pep_mass,precursor_intensity,protein
0,20170531_ETNP_JA14_01.31180.31180.2,5.520,1.000,2.290000e-11,28/44,R.AIQQQIENPLAQQILSGELVPGK.V,2473.3540,1515230.0,gi|54036848|sp|P63284.1|CLPB_ECOLI
1,20170531_ETNP_JA14_01.31115.31115.3,5.409,0.497,3.400000e-07,27/88,R.AIQQQIENPLAQQILSGELVPGK.V,2473.3540,4964440.0,gi|54036848|sp|P63284.1|CLPB_ECOLI
2,20170531_ETNP_JA14_01.17735.17735.2,4.315,0.430,3.940000e-06,26/36,R.LPQVEGTGGDVQPSQDLVR.V,1994.0069,150359000.0,gi|54036848|sp|P63284.1|CLPB_ECOLI
3,20170531_ETNP_JA14_01.22212.22212.3,3.774,0.350,1.640000e-04,21/60,R.VIGQNEAVDAVSNAIR.R,1654.8638,18102200.0,gi|54036848|sp|P63284.1|CLPB_ECOLI
4,20170531_ETNP_JA14_01.22093.22093.2,3.566,0.426,1.350000e-05,20/30,R.VIGQNEAVDAVSNAIR.R,1654.8638,31496000.0,gi|54036848|sp|P63284.1|CLPB_ECOLI


In [7]:
# get rid of rows where the xcorr is unavailable (usually 3 or so)
comet243 = comet243[comet243.xcorr != '[unavailable]']

# use str.strip with indexing by str[0] to add a column with the peptide's left terminus
comet243['L terminus'] = comet243['peptide'].astype(str).str[0]

# use str.strip with indexing by str[-1] to add a column with the peptide's left terminus
comet243['R terminus'] = comet243['peptide'].str.strip().str[-1]

# use a count function to enumerate the # of A's (alanines) in each peptide
comet243['A'] = comet243['peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
comet243['C'] = comet243['peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
comet243['D'] = comet243['peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
comet243['E'] = comet243['peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
comet243['F'] = comet243['peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
comet243['G'] = comet243['peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
comet243['H'] = comet243['peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in comet243 output, there will be no isoleucines (they're lumped in with leucines)
comet243['I'] = comet243['peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
comet243['K'] = comet243['peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
comet243['L'] = comet243['peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
comet243['M'] = comet243['peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
comet243['N'] = comet243['peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
comet243['P'] = comet243['peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
comet243['Q'] = comet243['peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
comet243['R'] = comet243['peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
comet243['S'] = comet243['peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
comet243['T'] = comet243['peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
comet243['V'] = comet243['peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
comet243['W'] = comet243['peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
comet243['Y'] = comet243['peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
comet243['c-carb'] = comet243['peptide'].str.count("160.03")

# use a count function to enumerate the # of oxidized M's in each peptide
comet243['m-oxid'] = comet243['peptide'].str.count("147.04")

# use a count function to enumerate the # of deamidated N's in each peptide
comet243['n-deam'] = comet243['peptide'].str.count("115.03")

# use a count function to enumerate the # of deamidated Q's in each peptide
comet243['q-deam'] = comet243['peptide'].str.count("129.04")

# use a count function to enumerate the # of hydroxylated K's in each peptide
comet243['k-hydr'] = comet243['peptide'].str.count("144.09")

# use a count function to enumerate the # of hydroxylated P's in each peptide
# I removed P hydroxyation in final searches because there were so few
#comet243['p-hydr'] = comet243['peptide'].str.count("131.05")

# use a count function to enumerate the # of methylated R's in each peptide
comet243['r-meth'] = comet243['peptide'].str.count("170.12")

# create a column with 'stripped' peptide sequences using strip
comet243['stripped peptide'] = comet243['peptide'].str[2:].str[:-2].str.replace(r"\[.*\]","")

# add a column with the stripped peptide length (number of AAs)
comet243['stripped length'] = comet243['stripped peptide'].apply(len)

comet243['NAAF num.'] = comet243['precursor_intensity'] / comet243['stripped length']

# total the number of modifications in sequence
comet243['ptm-total'] = comet243['c-carb'] + comet243['m-oxid'] + comet243['n-deam'] + comet243['q-deam'] + comet243['k-hydr'] + comet243['r-meth']

# write modified dataframe to new txt file
comet243.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/243/ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_Comet15_fungidb.csv")

# check out the results
comet243.head()

/home/millieginty/anaconda3/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


,spectrum,xcorr,deltacn,expect,ions,peptide,calc_neutral_pep_mass,precursor_intensity,protein,L terminus,...,c-carb,m-oxid,n-deam,q-deam,k-hydr,r-meth,stripped peptide,stripped length,NAAF num.,ptm-total
0,20170531_ETNP_JA14_01.31180.31180.2,5.520,1.000,2.290000e-11,28/44,R.AIQQQIENPLAQQILSGELVPGK.V,2473.3540,1515230.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,0,0,0,0,AIQQQIENPLAQQILSGELVPGK,23,6.587957e+04,0
1,20170531_ETNP_JA14_01.31115.31115.3,5.409,0.497,3.400000e-07,27/88,R.AIQQQIENPLAQQILSGELVPGK.V,2473.3540,4964440.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,0,0,0,0,AIQQQIENPLAQQILSGELVPGK,23,2.158452e+05,0
2,20170531_ETNP_JA14_01.17735.17735.2,4.315,0.430,3.940000e-06,26/36,R.LPQVEGTGGDVQPSQDLVR.V,1994.0069,150359000.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,0,0,0,0,LPQVEGTGGDVQPSQDLVR,19,7.913632e+06,0
3,20170531_ETNP_JA14_01.22212.22212.3,3.774,0.350,1.640000e-04,21/60,R.VIGQNEAVDAVSNAIR.R,1654.8638,18102200.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,0,0,0,0,VIGQNEAVDAVSNAIR,16,1.131388e+06,0
4,20170531_ETNP_JA14_01.22093.22093.2,3.566,0.426,1.350000e-05,20/30,R.VIGQNEAVDAVSNAIR.R,1654.8638,31496000.0,gi|54036848|sp|P63284.1|CLPB_ECOLI,R,...,0,0,0,0,0,0,VIGQNEAVDAVSNAIR,16,1.968500e+06,0


## Calculating the false discovery rate (% FDR)

### Filtering PSMs > a selected XCorr value and exporting peptides

In [8]:
# Let's separate out the decoy hits from the good ones

comet243pmm = comet243[~comet243['protein'].str.contains("DECOY")]
comet243dec = comet243[comet243['protein'].str.contains("DECOY")]

# how many PSM that are only PMM (proteins in the database)?

print("# real Comet PSMs", len(comet243pmm))

# compared to how many PSMs containing decoys?

print("# decoy Comet PSMs", len(comet243dec))

# calculate the bulk FDR (all PSMs so let's not beat ourselves up)

r = len(comet243pmm)
d = len(comet243dec)

FDR = d/r*100

print("False discovery rate = ", FDR)

# real Comet PSMs 6895
# decoy Comet PSMs 6416
False discovery rate =  93.05293691080493


In [14]:
# keep only peptides  >3 XCorr
# need to convert Xcorr column from strings to numeric so we can use loc
# note that pmm here just means 'non decoy', it's a vestige from the accession prefixes from the MED proteins
comet243['xcorr'] = pd.to_numeric(comet243['xcorr'])

comet243_3 = comet243.loc[comet243['xcorr'] >= 2.36]

# What's the FDR?

# Let's separate out the decoy hits from the good ones

comet243pmm3 = comet243_3[~comet243_3['protein'].str.contains("DECOY")]
comet243dec3 = comet243_3[comet243_3['protein'].str.contains("DECOY")]

# how many PSM that are only PMM (proteins in the database)?

print("# real Comet PSMs", len(comet243pmm3))

# compared to how many PSMs containing decoys?

print("# decoy Comet PSMs", len(comet243dec3))

# calculate the FDR 

r = len(comet243pmm3)
d = len(comet243dec3)

FDR = d/(d+r)*100

print("False discovery rate = ", FDR)

# real Comet PSMs 36
# decoy Comet PSMs 0
False discovery rate =  0.0


### Exporting txt files of stripped peptides at confidence cutoffs:

In [15]:
# keep only peptide column for the 1% FDR filtered PSMs
pep243moddup = comet243pmm3[["peptide"]]

# keep only the stripped peptide column for the 1% FDR filtered PSMs
# this is what we'll use for UniPept input, etc
pep243dup = comet243pmm3[["stripped peptide"]]

# deduplicate both of these lists
pep243mod = pep243moddup.drop_duplicates()
pep243 = pep243dup.drop_duplicates()

# print out the #s of modified and stripped peptides, deduplicated and not

print("Total modified peptides in 243:", len(pep243moddup))
print("Deduplicated modified peptides in 243:", len(pep243mod))
print("Total strippled peptides in 243:", len(pep243dup))
print("Deduplicated stripped peptides in 243:", len(pep243))

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing
pep243.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/243/ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_Comet15_1FDR_stripped_peptides.txt", header=False, index=False)

# made the text file into a FASTA 
!awk '{print ">"NR"\n"$0}' /home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/243/ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_Comet15_1FDR_stripped_peptides.txt > \
/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/243/ETNP-SKQ17-243-965m-0.3-JA14_PTM-opt_Comet15_1FDR_stripped_peptides.fas

# look at the stripped peptides
pep243.head()

Total modified peptides in 243: 36
Deduplicated modified peptides in 243: 27
Total strippled peptides in 243: 36
Deduplicated stripped peptides in 243: 25


,stripped peptide
0,AIQQQIENPLAQQILSGELVPGK
2,LPQVEGTGGDVQPSQDLVR
3,VIGQNEAVDAVSNAIR
6,PKEEEVGLDIAQNGER
9,VTDAEIAEVLAR


## NAAF correction and exporting files with AA and PTM totals:

### Only for DECOY-filtered Comet PSMs > XCorr 3.0

In [17]:
# made a new dataframe that contains the sums of certain columns in the modified
# peptide dataframe above (for >XCorr 3)

index = ['sample total']

data = {'A': comet243pmm3['A'].sum(),
        'C': comet243pmm3['C'].sum(),
        'D': comet243pmm3['D'].sum(),
        'E': comet243pmm3['E'].sum(),
        'F': comet243pmm3['F'].sum(),
        'G': comet243pmm3['G'].sum(),
        'H': comet243pmm3['H'].sum(),
        'I': comet243pmm3['I'].sum(),
        'K': comet243pmm3['K'].sum(),
        'L': comet243pmm3['L'].sum(),
        'M': comet243pmm3['M'].sum(),
        'N': comet243pmm3['N'].sum(),
        'P': comet243pmm3['P'].sum(),
        'Q': comet243pmm3['Q'].sum(),
        'R': comet243pmm3['R'].sum(),
        'S': comet243pmm3['S'].sum(),
        'T': comet243pmm3['T'].sum(),
        'V': comet243pmm3['V'].sum(),
        'W': comet243pmm3['W'].sum(),
        'Y': comet243pmm3['Y'].sum(),
        'c-carb': comet243pmm3['c-carb'].sum(),
        'm-oxid': comet243pmm3['m-oxid'].sum(),
        'n-deam': comet243pmm3['n-deam'].sum(),
        'q-deam': comet243pmm3['q-deam'].sum(),
        'k-hydr': comet243pmm3['k-hydr'].sum(),
        'r-meth': comet243pmm3['r-meth'].sum(),
        'Total area': comet243pmm3['precursor_intensity'].sum(),
        'Total length': comet243pmm3['stripped length'].sum()
       }

totalcomet243pmm3 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', 'q-deam', 'k-hydr', 'r-meth', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalcomet243pmm3['% C w/ carb'] = totalcomet243pmm3['c-carb'] / totalcomet243pmm3['C'] 

# calculate percentage of M's that are oxidized
totalcomet243pmm3['% M w/ oxid'] = totalcomet243pmm3['m-oxid'] / totalcomet243pmm3['M'] 

# calculate percentage of N's that are deamidated
totalcomet243pmm3['% N w/ deam'] = totalcomet243pmm3['n-deam'] / totalcomet243pmm3['N'] 

# calculate percentage of Q's that are deamidated
totalcomet243pmm3['% Q w/ deam'] = totalcomet243pmm3['q-deam'] / totalcomet243pmm3['Q'] 

# calculate percentage of K's that are hydroxylated
totalcomet243pmm3['% K w/ hydr'] = totalcomet243pmm3['k-hydr'] / totalcomet243pmm3['K'] 

# calculate percentage of P's that are hydroxylated
#totalcomet243pmm3['% P w/ hydr'] = totalcomet243pmm3['p-hydr'] / totalcomet243pmm3['P'] 

# calculate percentage of R's that are methylated
totalcomet243pmm3['% R w/ meth'] = totalcomet243pmm3['r-meth'] / totalcomet243pmm3['R'] 

# calculate NAAF denominator for all peptides in dataset i
totalcomet243pmm3['NAAF denom.'] = totalcomet243pmm3['Total area'] / totalcomet243pmm3['Total length']

# write modified dataframe to new txt file
totalcomet243pmm3.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/243/ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_Comet15_1FDR_totals.csv")

totalcomet243pmm3.head()

,A,C,D,E,F,G,H,I,K,L,...,r-meth,Total area,Total length,% C w/ carb,% M w/ oxid,% N w/ deam,% Q w/ deam,% K w/ hydr,% R w/ meth,NAAF denom.
sample total,70,0,40,52,6,56,4,42,18,39,...,1,444780720.0,536,NaN,0.25,0.095238,0.068182,0.0,0.02,829814.776119


In [18]:
# use the calculated NAAF factor (in totalcomet243pmm3 dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

NAAF3 = 829814.776119

# use NAAF >XCorr 3 to get NAAF
comet243pmm3['NAAF factor'] = (comet243pmm3['NAAF num.'])/NAAF3

# make a dataframe that contains only what we need: sequences, AAs, PTMs
comet243pmm3_NAAF = comet243pmm3[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', 'n-deam', \
                                'q-deam', 'k-hydr', 'r-meth']].copy()

# multiply the NAAF3 factor by the AAs to normalize its abundance by peak area and peptide length

comet243pmm3_NAAF['A-NAAF3'] = comet243pmm3_NAAF['A'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['C-NAAF3'] = comet243pmm3_NAAF['C'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['D-NAAF3'] = comet243pmm3_NAAF['D'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['E-NAAF3'] = comet243pmm3_NAAF['E'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['F-NAAF3'] = comet243pmm3_NAAF['F'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['G-NAAF3'] = comet243pmm3_NAAF['G'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['H-NAAF3'] = comet243pmm3_NAAF['H'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['I-NAAF3'] = comet243pmm3_NAAF['I'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['K-NAAF3'] = comet243pmm3_NAAF['K'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['L-NAAF3'] = comet243pmm3_NAAF['L'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['M-NAAF3'] = comet243pmm3_NAAF['M'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['N-NAAF3'] = comet243pmm3_NAAF['N'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['P-NAAF3'] = comet243pmm3_NAAF['P'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['Q-NAAF3'] = comet243pmm3_NAAF['Q'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['R-NAAF3'] = comet243pmm3_NAAF['R'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['S-NAAF3'] = comet243pmm3_NAAF['S'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['T-NAAF3'] = comet243pmm3_NAAF['T'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['V-NAAF3'] = comet243pmm3_NAAF['V'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['W-NAAF3'] = comet243pmm3_NAAF['W'] * comet243pmm3['NAAF factor']
comet243pmm3_NAAF['Y-NAAF3'] = comet243pmm3_NAAF['Y'] * comet243pmm3['NAAF factor']

# multiply the NAAF3 factor by the PTMs normalize its abundance by peak area and peptide length

comet243pmm3_NAAF['ccarb-NAAF3'] = comet243pmm3_NAAF['c-carb'] * comet243pmm3_NAAF['NAAF factor']
comet243pmm3_NAAF['moxid-NAAF3'] = comet243pmm3_NAAF['m-oxid'] * comet243pmm3_NAAF['NAAF factor']
comet243pmm3_NAAF['ndeam-NAAF3'] = comet243pmm3_NAAF['n-deam'] * comet243pmm3_NAAF['NAAF factor']
comet243pmm3_NAAF['qdeam-NAAF3'] = comet243pmm3_NAAF['q-deam'] * comet243pmm3_NAAF['NAAF factor']
comet243pmm3_NAAF['khydr-NAAF3'] = comet243pmm3_NAAF['k-hydr'] * comet243pmm3_NAAF['NAAF factor']
comet243pmm3_NAAF['rmeth-NAAF3'] = comet243pmm3_NAAF['r-meth'] * comet243pmm3_NAAF['NAAF factor']

# write the dataframe to a new csv
comet243pmm3_NAAF.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/243/ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_Comet15_1FDR_NAAF.csv")

comet243pmm3_NAAF.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,I,...,T-NAAF3,V-NAAF3,W-NAAF3,Y-NAAF3,ccarb-NAAF3,moxid-NAAF3,ndeam-NAAF3,qdeam-NAAF3,khydr-NAAF3,rmeth-NAAF3
0,AIQQQIENPLAQQILSGELVPGK,0.079391,2,0,0,2,0,2,0,3,...,0.000000,0.158781,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AIQQQIENPLAQQILSGELVPGK,0.260113,2,0,0,2,0,2,0,3,...,0.000000,0.520225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,LPQVEGTGGDVQPSQDLVR,9.536624,0,0,2,1,0,3,0,0,...,9.536624,38.146496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,VIGQNEAVDAVSNAIR,1.363422,3,0,1,1,0,1,0,2,...,0.000000,4.090265,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,VIGQNEAVDAVSNAIR,2.372216,3,0,1,1,0,1,0,2,...,0.000000,7.116648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# made a new dataframe that contains the sums of NAAF normalized AAs for comet243 Comet results
# also contains the sums of the NAAF-corrected PTMs occurances for each affected residue

index = ['sample total']

data = {'NAAF': comet243pmm3_NAAF['NAAF factor'].sum(),
        'A-NAAF': comet243pmm3_NAAF['A-NAAF3'].sum(),
        'C-NAAF': comet243pmm3_NAAF['C-NAAF3'].sum(),
        'D-NAAF': comet243pmm3_NAAF['D-NAAF3'].sum(),
        'E-NAAF': comet243pmm3_NAAF['E-NAAF3'].sum(),
        'F-NAAF': comet243pmm3_NAAF['F-NAAF3'].sum(),
        'G-NAAF': comet243pmm3_NAAF['G-NAAF3'].sum(),
        'H-NAAF': comet243pmm3_NAAF['H-NAAF3'].sum(),
        'I-NAAF': comet243pmm3_NAAF['I-NAAF3'].sum(),
        'K-NAAF': comet243pmm3_NAAF['K-NAAF3'].sum(),
        'L-NAAF': comet243pmm3_NAAF['L-NAAF3'].sum(),
        'M-NAAF': comet243pmm3_NAAF['M-NAAF3'].sum(),
        'N-NAAF': comet243pmm3_NAAF['N-NAAF3'].sum(),
        'P-NAAF': comet243pmm3_NAAF['P-NAAF3'].sum(),
        'Q-NAAF': comet243pmm3_NAAF['Q-NAAF3'].sum(),
        'R-NAAF': comet243pmm3_NAAF['R-NAAF3'].sum(),
        'S-NAAF': comet243pmm3_NAAF['S-NAAF3'].sum(),
        'T-NAAF': comet243pmm3_NAAF['T-NAAF3'].sum(),
        'V-NAAF': comet243pmm3_NAAF['V-NAAF3'].sum(),
        'W-NAAF': comet243pmm3_NAAF['W-NAAF3'].sum(),
        'Y-NAAF': comet243pmm3_NAAF['Y-NAAF3'].sum(),
        'C-carb-NAAF': comet243pmm3_NAAF['ccarb-NAAF3'].sum(),
        'M-oxid-NAAF': comet243pmm3_NAAF['moxid-NAAF3'].sum(),
        'N-deam-NAAF': comet243pmm3_NAAF['ndeam-NAAF3'].sum(),
        'Q-deam-NAAF': comet243pmm3_NAAF['qdeam-NAAF3'].sum(),
        'K-hydr-NAAF': comet243pmm3_NAAF['khydr-NAAF3'].sum(),
        'R-meth-NAAF': comet243pmm3_NAAF['rmeth-NAAF3'].sum()
       }

totalcomet243pmm3_NAAF = pd.DataFrame(data, columns=['NAAF', 'A-NAAF', 'C-NAAF', 'D-NAAF', 'E-NAAF', 'F-NAAF', \
                                                   'G-NAAF', 'H-NAAF', 'I-NAAF','K-NAAF', 'L-NAAF', 'M-NAAF', \
                                                   'N-NAAF', 'P-NAAF', 'Q-NAAF', 'R-NAAF', 'S-NAAF', \
                                                   'T-NAAF', 'V-NAAF', 'W-NAAF', 'Y-NAAF', 'C-carb-NAAF', \
                                                   'M-oxid-NAAF', 'N-deam-NAAF', 'Q-deam-NAAF', 'K-hydr-NAAF',\
                                                   'R-meth-NAAF'], index=index)

# calculate the NAAF-corrected % modified C, M, N, Q, K, and Rs


totalcomet243pmm3_NAAF['% C w/ carb. NAAF'] = totalcomet243pmm3_NAAF['C-carb-NAAF'] / totalcomet243pmm3_NAAF['C-NAAF']
totalcomet243pmm3_NAAF['% M w/ oxid. NAAF'] = totalcomet243pmm3_NAAF['M-oxid-NAAF'] / totalcomet243pmm3_NAAF['M-NAAF']
totalcomet243pmm3_NAAF['% N w/ deam. NAAF'] = totalcomet243pmm3_NAAF['N-deam-NAAF'] / totalcomet243pmm3_NAAF['N-NAAF']
totalcomet243pmm3_NAAF['% Q w/ deam. NAAF'] = totalcomet243pmm3_NAAF['Q-deam-NAAF'] / totalcomet243pmm3_NAAF['Q-NAAF']
totalcomet243pmm3_NAAF['% K w/ hydr. NAAF'] = totalcomet243pmm3_NAAF['K-hydr-NAAF'] / totalcomet243pmm3_NAAF['K-NAAF']
totalcomet243pmm3_NAAF['% R w/ meth. NAAF'] = totalcomet243pmm3_NAAF['R-meth-NAAF'] / totalcomet243pmm3_NAAF['R-NAAF']

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalcomet243pmm3_NAAF['NAAF check'] = totalcomet243pmm3_NAAF['NAAF'] / 829814.776119

# write modified dataframe to new txt file, same name + totals
totalcomet243pmm3_NAAF.to_csv("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/TPP-PTMopt/ETNP-SKQ17-TPP-PTM-opt_15ppm_FUNGI/243/ETNP-SKQ17-243-965m-0.3-JA14_PTMopt_Comet15_1FDR_NAAF_totals.csv")

totalcomet243pmm3_NAAF.head()

,NAAF,A-NAAF,C-NAAF,D-NAAF,E-NAAF,F-NAAF,G-NAAF,H-NAAF,I-NAAF,K-NAAF,...,Q-deam-NAAF,K-hydr-NAAF,R-meth-NAAF,% C w/ carb. NAAF,% M w/ oxid. NAAF,% N w/ deam. NAAF,% Q w/ deam. NAAF,% K w/ hydr. NAAF,% R w/ meth. NAAF,NAAF check
sample total,35.078931,56.561526,0.0,48.70468,45.469195,0.20749,52.970739,0.177001,36.661526,21.587022,...,0.152667,0.0,0.012694,NaN,0.01276,0.002806,0.003499,0.0,0.000244,0.000042


### Using BioPython to query peptide sequences

I installed the BioPython package using `pip install biopython`. All instructions and information [here](https://www.tutorialspoint.com/biopython/index.htm). 

GitHub project: https://github.com/biopython/biopython

I'm relying on the ProtParam module to parse sequences for relative AA composition, instability, secondary structure, instability, and hydrophobicity. You can read more about that module and the studies the indecies are derived from here:

https://biopython.org/wiki/ProtParam

In [ ]:
# Bio.SeqIO is the standard Sequence Input/Output interface for BioPython 1.43 and later
# Bio.SeqIO provides a simple uniform interface to input and output assorted sequence file formats.
# (including multiple sequence alignments), but will only deal with sequences as SeqRecord objects

# for accepted file formats see https://biopython.org/wiki/SeqIO

from Bio import SeqIO
#for seq_record in SeqIO.parse("/home/millieginty/Documents/git-repos/2017-etnp/data/MED4/MED2_tryp_1raw_db_peptides_nmod.fasta", "fasta"):
    #print(seq_record.id)
    #print(repr(seq_record.seq))
    #print(len(seq_record))
    
# I commented the print functions out so the output doesn't take up too much space. 

In [ ]:
# seeing what the ProtParam module can do with a single protein sequence:

from Bio.SeqUtils.ProtParam import ProteinAnalysis

test_seq = "MAEGEITTFTALTEKFNLPPGNYKKPKLLYCSNGGHFLRILPDGTVDGTRDRSDQHIQLQLSAESVGEVYIKSTETGQYLAMDTSGLLYGSQTPSEECLFLERLEENHYNTYTSKKHAEKNWFVGLKKNGSCKRGPRTHYGQKAILFLPLPV"

analysed_seq = ProteinAnalysis(test_seq)
print("molecular weight of seq =", analysed_seq.molecular_weight())

# calculates the aromaticity value of a protein according to Lobry & Gautier (1994, Nucleic Acids Res., 22, 3174-3180). 
# it's simply the relative frequency of Phe+Trp+Tyr.

analysed_seq.aromaticity()
print("aromaticity of seq =", analysed_seq.aromaticity())

# secondary_structure_fraction:
# this methods returns a list of the fraction of amino acids which tend to be in helix, turn or sheet. 
# AAs in helix: V, I, Y, F, W, L
# AAs in turn: N, P, G, S
# AAs in sheet: E, M, A, L
# the returned list contains 3 values: [Helix, Turn, Sheet]

analysed_seq.secondary_structure_fraction()
print("frac in H T S =", analysed_seq.secondary_structure_fraction())

# the instability index, an implementation of the method of Guruprasad et al. (1990, Protein Engineering, 4, 155-161).
# this method tests a protein for stability. 
# any value above 40 means the protein is unstable (=has a short half life)
# NOT SURE WHAT THIS MEANS FOR PEPTIDES, BUT WE COULD DO THIS FOR PROTEINS

analysed_seq.instability_index()
print("instability =", analysed_seq.instability_index())

# count_amino_acids will do just that, and get_amino_acids_percent will return %'s for each AA across the sequence. 
analysed_seq.get_amino_acids_percent()

# taking the returned dictionary and converting to a dataframe

aadict = analysed_seq.get_amino_acids_percent()
aadf = pd.DataFrame(list(aadict.items()),columns = ['residue','% occurance']) 

aadf.head()

In [ ]:
# use SeqIO and a loop to apply count_amino_acids to each sequence in the file
# aatot will give us the total number of each residue in the entire sample output

import collections
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis

all_aas = collections.defaultdict(int)
for record in SeqIO.parse("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas", "fasta"):
    x = ProteinAnalysis(str(record.seq))
    #print(record.id, x.count_amino_acids())
    for aa, count in x.count_amino_acids().items():
        all_aas[aa] += count        
        
# made a dataframe for amino acid total counts        
data = (all_aas)
aatot = pd.DataFrame(data, index = ['sample sequence total'])
aatot.head()

In [ ]:
from pandas import Series, DataFrame

with open('/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas') as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        
        
#converting lists to pandas Series    
s1 = Series(identifiers, name='ID')
s2 = Series(lengths, name='length')

#Gathering Series into a pandas DataFrame and rename index as ID column
idseq = DataFrame(dict(ID=s1, length=s2)).set_index(['ID'])

idseq.head()

In [ ]:
from pandas import Series, DataFrame



with open('/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas) as fasta_file:  # Will close handle cleanly
    identifiers = []
    lengths = []
    aa = []
    for seq_record in SeqIO.parse(fasta_file, 'fasta'):  # (generator)
        identifiers.append(seq_record.id)
        lengths.append(len(seq_record.seq))
        aa.count_amino_acids(seq_record.seq)
        
        
#converting lists to pandas Series    
s1 = Series(identifiers, name='ID')
s2 = Series(lengths, name='length')
s3 = Series(aa, name='AAs')

#Gathering Series into a pandas DataFrame and rename index as ID column
idseq = DataFrame(dict(ID=s1, length=s2, AAs=s3)).set_index(['ID'])

idseq.head()

In [ ]:
from Bio import SeqIO
from Bio.SeqUtils import ProtParam

handle = open("/home/millieginty/Documents/git-repos/2017-etnp/data/pro2020/ETNP-SKQ17/PEAKS-PTMopt/ETNP-SKQ17-231-100m-0.3-JA2_DN50_stripped_peptides.fas") 
for record in SeqIO.parse(handle, "fasta"): 
    seq = str(record.seq)
    X = ProtParam.ProteinAnalysis(seq)
    print(X.count_amino_acids()) 
    #print X.get_amino_acids_percent() 
    #print X.molecular_weight() 
    #print X.aromaticity() 
    #print X.instability_index() 
    #print X.flexibility() 
    #print X.isoelectric_point() 
    #print X.secondary_structure_fraction()
    
# made a data series from the count_amino_acids function
# aacount = {X.count_amino_acids()}

# made a pandas dataframe from the series generated above
# aacount = pd.DataFrame(list(data.items()),columns = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']) 

aacount = pd.DataFrame(X.count_amino_acids(), index=[0])

# look at new dataframe

# aacount.head()